**<font color=red> AMACIMIZ**

Elimizde filmler ile alakalı bir veri seti var.Bu veri setini okuyarak film izleyen bir kullanıcının, izlediği filmlerin özelliklerine bakarak kullanıcıya benzer özellikte yeni filmleri tavsiye edeceğiz.

In [1]:
import pandas as pd
import numpy as np 
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
df = pd.read_csv("tmdb_5000_movies.csv")
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [3]:
# Movies verimizin sütunlarını görmek için
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [4]:
# sadece kullanacaağımız kolonları görelim.
df = df[["original_title","overview"]]
df.head()

,original_title,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca..."


**<font color=red>İÇERİK TABANLI ÖNERİ SİSTEMİ - CONTENT-BASE RECOMMENDATION SYSTEM**

In [5]:
# overview: Bir filmin genel özelliklerini, ana hatlarını veya ana noktalarını içeren kısa bir açıklama anlamına gelir.bu sutundaki bilgileri kullanarak filmlerin birbirlerine olan benzerliklerini oluşturacağız.

In [6]:
#Film hakkında önizleme yazılarını listeliyoruz.
df["overview"].head()

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [7]:
# eksik değerler var mı kontrol ediyoruz.
df.overview.isnull().sum()

3

In [8]:
# 3 tane eksik verimiz var bu verileri silebiliriz yada "" ifadesini kullanarak boş olacak şekilde doldurabiliriz.ben bu şekilde  doldurmayı tercih ediyorum.

df["overview"] = df["overview"].fillna("")
df.overview.isnull().sum()

0

In [9]:
# Filmlerimizi tanımlayan kelimeleri ve sıklıklarını hesaplıyoruz.Benzerliklerini anlayabilmemiz için Matrix dönüşümü uyguluyoruz

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df["overview"])

**<font color=red>COSINE BENZERLİĞİNİ KULLANMAK**

Projemizde Cosine Benzerliğini kullanarak benzerlik puanlarımızı bulacağız

In [11]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]])

In [12]:
# indexlerin ve film başlıklarının ters haritasını oluşturuyoruz. Tekrar eden değerleri listeden çıkarıyoruz
indices = pd.Series(df.index, index=df["original_title"]).drop_duplicates()
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [13]:
# Bu fonksiyon ile fonksiyona göndermiş olduğumuz filme en yakın benzerlikte ki 10 filmi geri döndürecek

def get_recommendations(title, cosine_sim=cosine_sim):
    # Girilen filmin indeksini al
    idx = indices[title]

    # Bu filmin diğer tüm filmlerle olan benzerlik puanlarını listele
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Filmleri benzerlik puanlarına göre sırala
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # En çok benzeyen 10 filmin skorunu al
    sim_scores = sim_scores[1:11]

    # Film indekslerini al
    movie_indices = [i[0] for i in sim_scores]

    # En çok benzeyen 10 filmi döndür
    return df['original_title'].iloc[movie_indices]


**<font color=red>ÖNERİ FONKSİYONUMUZ TAMAM. ÖRNEK YAPALIM**

In [14]:
#The Dark Knight içeriğine benzer 10 film önerisi
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: original_title, dtype: object

In [15]:
get_recommendations('The Godfather')

2731     The Godfather: Part II
1873                 Blood Ties
867     The Godfather: Part III
3727                Snabba Cash
3623                       Made
3125                     Eulogy
3896                   Sinister
4506            The Maid's Room
3783                        Joe
2244      The Cold Light of Day
Name: original_title, dtype: object